In [1]:
# https://stackoverflow.com/questions/43599018/is-there-a-way-to-get-matplotlib-path-contains-points-to-be-inclusive-of-boundar
#I do quite like this command in Jupiter notebook:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
#It makes things wider and not waste the space on your screen
import pandas as pd
import importlib
import csv,os
import numpy as np
import pandas as pd
import itertools
import pyproj
import shapely
import copy
from uafgi import gicollections,cfutil,glacier,gdalutil,shputil
import uafgi.data.ns642
import netCDF4
import importlib

In [2]:

select = pd.read_pickle('select_03.df')

In [3]:
row = select.iloc[0]
list(row.keys())


['w21_popular_name',
 'w21_greenlandic_name',
 'w21_coast',
 'w21_category',
 'w21_Qr',
 'w21_Qf',
 'w21_Qm',
 'w21_Qs',
 'w21_Qc_inferred',
 'w21_qm',
 'w21_qf',
 'w21_qc',
 'w21_mean_depth',
 'w21_min_depth',
 'w21_quality_str',
 'w21_area_grounded_1992_2017',
 'w21_area_grounded_1992_1997',
 'w21_area_grounded_1998_2007',
 'w21_area_grounded_2008_2017',
 'w21_mean_fjord_width',
 'w21_length_grounded_1992_2017',
 'w21_length_grounded_1992_1997',
 'w21_length_grounded_1998_2007',
 'w21_length_grounded_2008_2017',
 'w21_ocean_model_sample_area',
 'w21_mean_TF_1992-2017',
 'w21_mean_TF_1992-1997',
 'w21_mean_TF_1998-2007',
 'w21_mean_TF_2008-2017',
 'w21_subglacial_discharge_1992_2017',
 'w21_subglacial_discharge_1992_1997',
 'w21_subglacial_discharge_1998_2007',
 'w21_subglacial_discharge_2008_2017',
 'w21_mean_xsection_area',
 'w21_mean_undercutting_1992_2017',
 'w21_mean_undercutting_1992_1997',
 'w21_mean_undercutting_1998_2007',
 'w21_mean_undercutting_2008_2017',
 'w21_mean_underc

In [4]:
fjc = row['fjord_classes']
fjord = np.isin(fjc, glacier.ALL_FJORD)
grid = row['ns481_grid']

In [5]:
grid

'E71.75N'

In [6]:
importlib.reload(gdalutil)

itslive_nc = 'outputs/itslive/GRE_G0240_{}_2011_2018.nc'.format(grid)
grid_info = gdalutil.FileInfo(itslive_nc)
grid_info.x

Axis(centers=array([490000., 490100., 490200., 490300., 490400., 490500., 490600.,
       490700., 490800., 490900., 491000., 491100., 491200., 491300.,
       491400., 491500., 491600., 491700., 491800., 491900., 492000.,
       492100., 492200., 492300., 492400., 492500., 492600., 492700.,
       492800., 492900., 493000., 493100., 493200., 493300., 493400.,
       493500., 493600., 493700., 493800., 493900., 494000., 494100.,
       494200., 494300., 494400., 494500., 494600., 494700., 494800.,
       494900., 495000., 495100., 495200., 495300., 495400., 495500.,
       495600., 495700., 495800., 495900., 496000., 496100., 496200.,
       496300., 496400., 496500., 496600., 496700., 496800., 496900.,
       497000., 497100., 497200., 497300., 497400., 497500., 497600.,
       497700., 497800., 497900., 498000., 498100., 498200., 498300.,
       498400., 498500., 498600., 498700., 498800., 498900., 499000.,
       499100., 499200., 499300., 499400., 499500., 499600., 499700.,
       

In [10]:
importlib.reload(cfutil)
with netCDF4.Dataset(itslive_nc) as nc:
    time = [dt.year for dt in cfutil.read_time(nc, 'time')]
    u_surface = nc.variables[u_ssa_bc]
    print(list(nc.variables))
time

['time', 'time_bnds', 'x', 'y', 'polar_stereographic', 'u_ssa_bc', 'v_ssa_bc', 'v']


[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [8]:
importlib.reload(glacier)


<module 'uafgi.glacier' from '/Users/eafischer2/harn/pismip6/uafgi/uafgi/glacier.py'>

In [33]:
print(row['up_loc'])

POINT (546206.5823260495 -1872217.376889129)


In [39]:
grid_info.x

Axis(centers=masked_array(data=[490000., 490100., 490200., 490300., 490400., 490500.,
                   490600., 490700., 490800., 490900., 491000., 491100.,
                   491200., 491300., 491400., 491500., 491600., 491700.,
                   491800., 491900., 492000., 492100., 492200., 492300.,
                   492400., 492500., 492600., 492700., 492800., 492900.,
                   493000., 493100., 493200., 493300., 493400., 493500.,
                   493600., 493700., 493800., 493900., 494000., 494100.,
                   494200., 494300., 494400., 494500., 494600., 494700.,
                   494800., 494900., 495000., 495100., 495200., 495300.,
                   495400., 495500., 495600., 495700., 495800., 495900.,
                   496000., 496100., 496200., 496300., 496400., 496500.,
                   496600., 496700., 496800., 496900., 497000., 497100.,
                   497200., 497300., 497400., 497500., 497600., 497700.,
                   497800., 497900., 4

In [48]:
grid_info.wkt

'PROJCS["WGS 84 / NSIDC Sea Ice Polar Stereographic North",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",70],PARAMETER["central_meridian",-45],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],AUTHORITY["EPSG","3413"]]'

In [62]:
ns642 = uafgi.data.ns642.read(grid_info.wkt)
ns642g = dict(list(ns642.df[['ns642_GlacierID', 'ns642_year0', 'ns642_terminus']].groupby(['ns642_GlacierID'])))
ns642g

{1:       ns642_GlacierID  ns642_year0  \
 0                   1         2000   
 205                 1         2005   
 408                 1         2006   
 608                 1         2007   
 1012                1         2008   
 1210                1         2012   
 1413                1         2014   
 1466                1         2015   
 1701                1         2016   
 
                                          ns642_terminus  
 0     LINESTRING (-216941.3759366392 -2300666.311248...  
 205   LINESTRING (-216871.9646593904 -2300620.927435...  
 408   LINESTRING (-216728.246110465 -2301030.4055218...  
 608   LINESTRING (-216752.7014791318 -2300739.281152...  
 1012  LINESTRING (-212306.700629078 -2298974.6543078...  
 1210  LINESTRING (-216950.0120601932 -2301082.707890...  
 1413  LINESTRING (-216937.4865989024 -2300890.914418...  
 1466  LINESTRING (-216826.2655000001 -2300805.6669, ...  
 1701  LINESTRING (-216690.0803880789 -2300692.258949...  ,
 2:       ns64

In [106]:
importlib.reload(glacier)
importlib.reload(gdalutil)
dfx = ns642g[row['ns642_GlacierID']].sort_values('ns642_year0')
years = dfx['ns642_year0'].to_list()
termini = dfx['ns642_terminus'].to_list()
ice_area = [glacier.ice_area(grid_info, fjord, row['up_loc'], t) for t in termini]
ice_len = [x / (row['w21_mean_fjord_width'] * 1000.) for x in ice_area]

def linReg(x, y):
    '''linear regression using numpy starting from two one dimensional numpy arrays'''
    A = np.vstack([x, np.ones(len(x))]).T
    slope, intercept = np.linalg.lstsq(A, y, rcond=None)[0]
    return pd.Series({'slope':slope, 'intercept': intercept})

linReg(years, ice_len)

slope           -62.792085
intercept    132682.889225
dtype: float64

In [98]:
importlib.reload(glacier)
t = termini[0]
a = glacier.ice_area(grid_info, fjord, row['up_loc'], t)
a

dx 100.0
dy 100.0
<class 'osgeo.osr.SpatialReference'>
1773
<class 'osgeo.osr.SpatialReference'>


17730000.0

In [76]:
glacier.ice_area

<function uafgi.glacier.ice_area(grid_info, fjord, up_loc, terminus)>

In [96]:
grid_info.dx

100.0

In [117]:
glacier_ids = set(select['ns642_GlacierID'].to_list())
ns642g = ns642.df[ns642.df['ns642_GlacierID'].isin(glacier_ids)]

,ns642_GlacierID,ns642_terminus,ns642_year0,ns642_year1,ns642_allnames,ns642_date,ns642_QualFlag,ns642_ImgSource,ns642_key
2,3,LINESTRING (-197627.3018625653 -2274508.533470...,2000,2001,"(Jakobshavn Isbrae, Sermeq Kujalleq, Sermeq Ku...","(2000-09-30 00:00:00, 2001-01-23 00:00:00)",NaN,NaN,"(3, 2001)"
7,8,LINESTRING (-205164.0096814953 -2172531.715912...,2000,2001,"(Sermeq avannarleq B, Sermeq Avannarleq B, Ser...","(2000-09-30 00:00:00, 2001-01-23 00:00:00)",NaN,NaN,"(8, 2001)"
8,9,LINESTRING (-210317.0961012484 -2132583.073936...,2000,2001,"(Store gletscher, Sermeq Kujalleq, Sermeq Kuja...","(2000-09-30 00:00:00, 2001-01-23 00:00:00)",NaN,NaN,"(9, 2001)"
11,12,LINESTRING (-208089.7902075725 -2096724.677272...,2000,2001,"(Kangilleq Kangigdleq Isbrae, Kangilleq, Kangi...","(2000-09-30 00:00:00, 2001-01-23 00:00:00)",NaN,NaN,"(12, 2001)"
13,14,LINESTRING (-214983.8014421279 -2064816.580707...,2000,2001,"(Perdlerfiup sermia, Perlerfiup Sermia, Perler...","(2000-09-30 00:00:00, 2001-01-23 00:00:00)",NaN,NaN,"(14, 2001)"
...,...,...,...,...,...,...,...,...,...
1867,193,LINESTRING (147195.7337052741 -2950173.7907092...,2016,2017,"(, Rimfaxe, Guldfaxe, )",2017-02-05 00:00:00,0.0,Sentinel1,"(193, 2017)"
1879,205,LINESTRING (-202607.3930545626 -3088107.404597...,2016,2017,"(Uukkaasorsuaq, Uukkaasorsuaq, Sermilik Bræ, U...",2017-02-03 00:00:00,0.0,Sentinel1,"(205, 2017)"
1882,207,LINESTRING (-229830.1325783396 -2820852.643145...,2016,2017,"(Kangiata Nunaata Sermia, Kangiata Nunaata Ser...",2017-02-03 00:00:00,2.0,Sentinel1,"(207, 2017)"
1918,175,LINESTRING (309871.7179047784 -2574875.3509310...,2016,2017,"(, Helheim Gletsjer, , )",2017-02-05 00:00:00,0.0,Sentinel1,"(175, 2017)"
